<a href="https://colab.research.google.com/github/ricardodeazambuja/mlflow-er/blob/main/TestingUsingGoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+https://github.com/ricardodeazambuja/mlflow-er.git --upgrade

In your local computer, launch the mlflow server:    
`mlflow server`

And launch ngrok:    
`ngrok http 5000`


Now, get the URL generated by ngrok (something like `https://888f-142-120-136-7.ngrok.io`) and replace in the cell below (`LOCAL_URI`).


To visualize your experiments, open in your browser the URL generated by mlflow server (if you used the defaults it will be `http://127.0.0.1:5000`).

In [13]:
%%file test.py

# the "magic" above will save the contents of this cell to a file called test.py

from time import sleep, time
from copy import deepcopy
import argparse
import os

import mlflow
from mlflow_er import ExperimentTracker

import numpy as np

LOCAL_URI = "https://888f-142-120-136-7.ngrok.io"


parser = argparse.ArgumentParser()
parser.add_argument("--background_logger", action='store_true')
args = parser.parse_args()

USE_BACKGROUND_LOGGER = args.background_logger

if USE_BACKGROUND_LOGGER:
    print("Using background logger!")
try:
    def do_cool_stuff(epoch, time_spent=5):
        init_time = time()
        cool_loss = np.random.rand(1)[0]
        current_model = np.random.rand(100, 100, 100).astype(dtype=np.float16)
        time_diff = time_spent - (time() - init_time)
        if time_diff>0:
            print(f"Sleeping for {time_diff:.3f}s to emulate time spent by the model doing cool stuff that mostly uses the GPU...")
            sleep(time_diff)
        return cool_loss, current_model


    my_experiment = ExperimentTracker("Cool Experiment", tracking_uri=LOCAL_URI)

    time_spent_logging = 0
    epochs = 10
    
    if USE_BACKGROUND_LOGGER:
        run_name = "BackgroundLogger"
        description = "Using Background Logger"
    else:
        run_name = "DefaultLogger"
        description = "Using Default Logger"
    with my_experiment.run(run_name=run_name, description=description) as run:
        prev_loss = float('inf')
        prev_model = None
        for epoch in range(epochs):
            print(f"Epoch #{epoch}")
            cool_loss, current_model = do_cool_stuff(epoch)
            print("Done do_cool_stuff!")
            
            prev_time = time()
            if USE_BACKGROUND_LOGGER:
                my_experiment.background_worker("cool_loss", mlflow.log_metric, key="cool_loss", value=cool_loss, step=epoch)
                print("Done cool_loss!")
            else:
                mlflow.log_metric("cool_loss", cool_loss, step=epoch)
            time_spent_logging += time() - prev_time

            if cool_loss < prev_loss:
                prev_loss = cool_loss
                prev_model = deepcopy(current_model)
        
        print(f"Saving model...")
        np.savez_compressed("best_model.npz", best_model=prev_model)
        prev_time = time()
        if USE_BACKGROUND_LOGGER:
            my_experiment.background_worker("best_model", mlflow.log_artifact, use_process=True, local_path="best_model.npz")
        else:
            mlflow.log_artifact(local_path="best_model.npz")
        time_spent_logging += time() - prev_time

    print(f"Total time spent with logging: {time_spent_logging}")

finally:
    if os.path.exists("best_model.npz"):
        os.remove("best_model.npz")

Overwriting test.py


First let's test using the background_logger:

In [12]:
!python test.py --background_logger

Using background logger!
fatal: not a git repository (or any of the parent directories): .git
Creating a new experiment (Cool Experiment) at https://4276-142-120-136-7.ngrok.io ... Done!
--
Experiment Name: Cool Experiment
Experiment ID: 231064903911896151
Tracking URI: https://4276-142-120-136-7.ngrok.io
Artifact Location: mlflow-artifacts:/231064903911896151
Experiment Tags: {'git_commit': ''}
Lifecycle Stage: active
Creation Timestamp: 1671491495815
Git commit: 
--
Cool Experiment (231064903911896151) - BackgroundLogger (f3b3aad1e78e401dbdfd7edc7722d564): RUNNING
Epoch #0
Sleeping for 4.983s to emulate time spent by the model doing cool stuff that mostly uses the GPU...
Done do_cool_stuff!
Done cool_loss!
Epoch #1
Sleeping for 4.980s to emulate time spent by the model doing cool stuff that mostly uses the GPU...
Done do_cool_stuff!
Done cool_loss!
Epoch #2
Sleeping for 4.983s to emulate time spent by the model doing cool stuff that mostly uses the GPU...
Done do_cool_stuff!
Done coo

and now without the background_logger:

In [14]:
!python test.py 

--
Experiment Name: Cool Experiment
Experiment ID: 231064903911896151
Tracking URI: https://888f-142-120-136-7.ngrok.io
Artifact Location: mlflow-artifacts:/231064903911896151
Experiment Tags: {'git_commit': ''}
Lifecycle Stage: active
Creation Timestamp: 1671491495815
Git commit: 
--
Cool Experiment (231064903911896151) - DefaultLogger (5d2171779192428d824be764795a17dc): RUNNING
Epoch #0
Sleeping for 4.982s to emulate time spent by the model doing cool stuff that mostly uses the GPU...
Done do_cool_stuff!
Epoch #1
Sleeping for 4.971s to emulate time spent by the model doing cool stuff that mostly uses the GPU...
Done do_cool_stuff!
Epoch #2
Sleeping for 4.985s to emulate time spent by the model doing cool stuff that mostly uses the GPU...
Done do_cool_stuff!
Epoch #3
Sleeping for 4.982s to emulate time spent by the model doing cool stuff that mostly uses the GPU...
Done do_cool_stuff!
Epoch #4
Sleeping for 4.985s to emulate time spent by the model doing cool stuff that mostly uses the